# Лабораторная работа №2 "Проведение исследований с логистической и линейной регрессией"

### Ход работы

Импортируем библиотеки перед работой

In [77]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import warnings

from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, KFold
from sklearn.linear_model import LogisticRegression, LinearRegression, ElasticNet, Ridge, Lasso
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error, mean_absolute_error, r2_score, log_loss
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.exceptions import ConvergenceWarning
from scipy.optimize import minimize
from sklearn.utils.extmath import softmax
from sklearn.base import BaseEstimator, RegressorMixin



##### Создание бейзлайна для модели классификации

Проведём те же манипуляции, что и ранне: выгрузим датасет и минимально его обработаем

In [5]:
c_base_df = pd.read_csv("../classification.csv").sample(frac=1, random_state=42).reset_index(drop=True)

c_base_df = c_base_df.drop(columns=['instance_id', 'obtained_date', 'track_name', 'artist_name'])

c_base_df.drop_duplicates()

c_base_df['tempo'] = pd.to_numeric(c_base_df['tempo'], errors='coerce')

le = LabelEncoder()
c_base_df['mode'] = le.fit_transform(c_base_df['mode'])
c_base_df['music_genre'] = le.fit_transform(c_base_df['music_genre'])
c_base_df['key'] = le.fit_transform(c_base_df['key'])

median_tempo = c_base_df['tempo'].median()
c_base_df['tempo'] = c_base_df['tempo'].fillna(median_tempo)

X_c_base = c_base_df.drop(columns=["music_genre"])
y_c_base = c_base_df["music_genre"]

X_c_base_train, X_c_base_test, y_c_base_train, y_c_base_test = train_test_split(
    X_c_base,
    y_c_base,
    test_size=0.2,
    random_state=42,
    stratify=y_c_base
)

Теперь обучим модель из sklearn

In [ ]:
warnings.filterwarnings("ignore", category=ConvergenceWarning)

model = LogisticRegression(max_iter=1000)

model.fit(X_c_base_train, y_c_base_train)

y_pred = model.predict(X_c_base_test)

acc = accuracy_score(y_c_base_test, y_pred)

print("Accuracy:", acc)

Accuracy: 0.3162


##### Улучшение бейзлайна для модели классификации

In [9]:
print(classification_report(y_c_base_test, y_pred))

              precision    recall  f1-score   support

           0       0.19      0.20      0.19      1000
           1       0.49      0.58      0.53      1000
           2       0.32      0.17      0.22      1000
           3       0.67      0.76      0.71      1000
           4       0.18      0.09      0.12      1000
           5       0.23      0.31      0.27      1000
           6       0.20      0.22      0.21      1000
           7       0.31      0.33      0.32      1000
           8       0.28      0.23      0.25      1000
           9       0.22      0.28      0.25      1000

    accuracy                           0.32     10000
   macro avg       0.31      0.32      0.31     10000
weighted avg       0.31      0.32      0.31     10000



Это явно лучше, чем KNN. Теперь снова применим техники из прошлой ЛР и улучшим модель

In [10]:
c_df = pd.read_csv("../classification.csv").sample(frac=1, random_state=42).reset_index(drop=True)

c_df = c_df.drop(columns=['instance_id', 'obtained_date', 'track_name', 'artist_name'])

pca = PCA(n_components=2)
c_df[['pc1', 'pc2']] = pca.fit_transform(c_df[['loudness', 'acousticness', 'energy']])
c_df = c_df.drop(columns=['loudness', 'acousticness', 'energy'])

c_df['tempo'] = pd.to_numeric(c_df['tempo'], errors='coerce')

le = LabelEncoder()
c_df['music_genre'] = le.fit_transform(c_df['music_genre'])
c_df['mode'] = le.fit_transform(c_df['mode'])

ohe = OneHotEncoder(sparse_output=False, drop='first')
encoded_key = ohe.fit_transform(c_df[['key']])
encoded_df_key = pd.DataFrame(encoded_key, columns=ohe.get_feature_names_out(['key']))
c_df = c_df.drop(columns=['key']).reset_index(drop=True)
c_df = pd.concat([c_df, encoded_df_key], axis=1)

c_df['duration_ms'] = c_df['duration_ms'].replace(-1, np.nan)

c_df['instrumental_flag'] = (c_df['instrumentalness'] > 0.05).astype(int)
c_df = c_df.drop(columns=['instrumentalness'])

c_df['undefined_tempo'] = c_df['tempo'].isna().astype(int)

median_tempo = c_df['tempo'].median()
c_df['tempo'] = c_df['tempo'].fillna(median_tempo)

median_duration = c_df['duration_ms'].median()
c_df['duration_ms'] = c_df['duration_ms'].fillna(median_duration)

float_features = [
    'popularity', 'danceability', 'duration_ms',
    'liveness', 'speechiness', 'tempo',
    'valence', 'pc1', 'pc2'
]

other_features = [
    'mode', 'instrumental_flag', 'undefined_tempo'
] + list(encoded_df_key.columns) 

X_c = c_df[float_features + other_features]
y_c = c_df['music_genre']

X_c_train, X_c_test, y_c_train, y_c_test = train_test_split(
    X_c, y_c, test_size=0.2, stratify=y_c, random_state=42
)

теперь перейдём к обучению

In [ ]:
preprocess = ColumnTransformer(
    transformers=[
        ('float_scaling', StandardScaler(), float_features),
    ],
    remainder='passthrough'
)

pipeline = Pipeline([
    ('preprocess', preprocess),
    ('logreg', LogisticRegression(max_iter=5000, random_state=42, n_jobs=-1))
])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    'logreg__C': [0.01, 0.1, 1, 10, 100],
    'logreg__penalty': ['l1', 'l2'],
    'logreg__class_weight': [None, 'balanced'],
    'logreg__solver': ['saga', 'liblinear']
}


grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

grid.fit(X_c_train, y_c_train)

print("Best parameters:", grid.best_params_)

best_model = grid.best_estimator_

y_c_pred = best_model.predict(X_c_test)
print("Score:", accuracy_score(y_c_test, y_c_pred))

print(classification_report(y_c_test, y_c_pred))

Fitting 5 folds for each of 40 candidates, totalling 200 fits
Best parameters: {'logreg__C': 0.1, 'logreg__class_weight': None, 'logreg__penalty': 'l1', 'logreg__solver': 'saga'}
Score: 0.5282
              precision    recall  f1-score   support

           0       0.38      0.28      0.32      1000
           1       0.62      0.63      0.63      1000
           2       0.50      0.43      0.47      1000
           3       0.76      0.79      0.78      1000
           4       0.46      0.58      0.51      1000
           5       0.56      0.60      0.58      1000
           6       0.48      0.52      0.50      1000
           7       0.49      0.42      0.45      1000
           8       0.47      0.38      0.42      1000
           9       0.51      0.65      0.57      1000

    accuracy                           0.53     10000
   macro avg       0.52      0.53      0.52     10000
weighted avg       0.52      0.53      0.52     10000



##### Создание бейзлайна для модели регрессии

Сделаем всё то же, что и ранее

In [60]:
r_base_df = pd.read_csv("../regression.csv").sample(frac=1, random_state=42).reset_index(drop=True)

r_base_df['Date'] = pd.to_datetime(r_base_df['Date'], dayfirst=True)

r_base_df["Year"] = r_base_df["Date"].dt.year
r_base_df["Month"] = r_base_df["Date"].dt.month
r_base_df["Day"] = r_base_df["Date"].dt.day

r_base_df = r_base_df.drop(columns=['Date'])

per_store_count = r_base_df.groupby('Store').size().iloc[0]
k = max(1, int(np.round(0.8 * per_store_count))) 
store_counts = r_base_df['Store'].nunique()

train = r_base_df.iloc[: store_counts * k]
test = r_base_df.iloc[store_counts * k :]

X_r_base_train = train.drop(columns=['Weekly_Sales'])
X_r_base_test = test.drop(columns=['Weekly_Sales'])
y_r_base_train = train['Weekly_Sales']
y_r_base_test = test['Weekly_Sales']

И теперь обучим модель

In [13]:
knn = LinearRegression()
knn.fit(X_r_base_train, y_r_base_train)
y_pred = knn.predict(X_r_base_test)

mae = mean_absolute_error(y_r_base_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_r_base_test, y_pred))
r2 = r2_score(y_r_base_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

MAE: 423673.5471083975
RMSE: 520176.70459250984
R2: 0.1636445664651608


##### Улучшение бейзлайна для модели регрессии

Сначала повторим техники из предыдущей ЛР

In [63]:
r_df = pd.read_csv("../regression.csv").sample(frac=1, random_state=42).reset_index(drop=True)

r_df['Date'] = pd.to_datetime(r_df['Date'], dayfirst=True)

r_df['Year'] = r_df['Date'].dt.year
r_df['Week'] = r_df['Date'].dt.isocalendar().week

r_df = r_df.drop(columns=['Date'])

Я пробовал играться с фичёй недели: делал сквозную нумерацию недель в рамках нескольких лет, добавлял синус и косинус недели, но лучшим вариантом в итоге было всё оставить как есть и применить к признаку OHE

Разобьём датасет.

In [64]:
num_features = ['Temperature', 'Fuel_Price', 'CPI', 'Unemployment', 'Year']
cat_store = ['Store', 'Week']
other_feats = ['Holiday_Flag']

per_store_count = r_df.groupby('Store').size().iloc[0]
k = max(1, int(np.round(0.8 * per_store_count))) 
store_counts = r_df['Store'].nunique()

train = r_df.iloc[: store_counts * k]
test = r_df.iloc[store_counts * k :]

X_r_train = train.drop(columns=['Weekly_Sales'])
X_r_test = test.drop(columns=['Weekly_Sales'])
y_r_train = train['Weekly_Sales']
y_r_test = test['Weekly_Sales']

Будем применять OHE для фич с номером магазина и НЕДЕЛИ, т.к. таргет не будет зависеть от них линейно. Нормализуем числовые фичи. Также применим кросс-валидацию и подбор гиперпараметров

In [ ]:
warnings.filterwarnings("ignore")

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_store),
        ('passth', 'passthrough', other_feats)
    ],
    remainder='drop'
)

pipeline = Pipeline([
    ('pre', preprocessor),
    ('model', ElasticNet(max_iter=20000, random_state=42))
])

param_grid = [
    {'model': [Ridge(max_iter=20000, random_state=42)],
     'model__alpha': np.logspace(-4, 2, 50)},
    {'model': [Lasso(max_iter=20000, random_state=42)],
     'model__alpha': np.logspace(-4, 1, 50)},
    {'model': [ElasticNet(max_iter=20000, random_state=42)],
     'model__alpha': np.logspace(-4, 1, 50),
     'model__l1_ratio': [0.1, 0.5, 0.9]}
]

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid.fit(X_r_train, y_r_train)

print("Best model:", grid.best_estimator_.named_steps['model'])
print("Best params:", grid.best_params_)

y_pred = grid.predict(X_r_test)

mae  = mean_absolute_error(y_r_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_r_test, y_pred))
r2   = r2_score(y_r_test, y_pred)

print(f"\nTest MAE:  {mae:.2f}")
print(f"Test RMSE: {rmse:.2f}")
print(f"Test R2:   {r2:.4f}")

Fitting 5 folds for each of 250 candidates, totalling 1250 fits
Best model: Lasso(alpha=np.float64(2.44205309454865), max_iter=20000, random_state=42)
Best params: {'model': Lasso(max_iter=20000, random_state=42), 'model__alpha': np.float64(2.44205309454865)}

Test MAE:  70758.16
Test RMSE: 109901.27
Test R2:   0.9627


Мы, опять же, сильно улучшили результат. Он даже по большинству метрик лучше чем у KNN

| Метрика | Улучшенная модель KNN | Бейзлайн линейной регрессии | Улучшенная линейная рергессия |
|-|-|-|-|
| MAE | 68212 | 432655 | 70758 |
| RMSE | 129073 | 518509 | 109901 |
| R2 | 0.949 | 0.129 | 0.963 |

##### Имплементация логистической регрессии

In [80]:
class MyLogisticRegression:
    """
    Многоклассовая логистическая регрессия с поддержкой L1/L2 регуляризации.
    Поддерживаются два метода оптимизации: 'lbfgs' и 'saga'.
    """

    def __init__(self, C=1.0, penalty='l2', solver='lbfgs', max_iter=1000, tol=1e-4):
        # Инициализация гиперпараметров
        self.C = C
        self.penalty = penalty
        self.solver = solver
        self.max_iter = max_iter
        self.tol = tol
        self.classes_ = None  # уникальные классы целевой переменной
        self.weights = None   # матрица весов модели

    def _one_hot(self, y):
        # Преобразование меток в one-hot представление
        y = np.array(y).reshape(-1, 1)
        encoder = OneHotEncoder(sparse_output=False, categories='auto')
        return encoder.fit_transform(y)

    def _loss_and_grad(self, W, X, Y_onehot):
        """
        Вычисление функции потерь (log-loss) и градиента с регуляризацией.
        """
        n_samples = X.shape[0]
        n_classes = Y_onehot.shape[1]
        W = W.reshape(n_classes, -1)
        scores = X.dot(W.T)
        probs = softmax(scores)
        loss = log_loss(Y_onehot, probs, normalize=True)

        # Регуляризация
        if self.penalty == 'l2':
            loss += 0.5 / (self.C * n_samples) * np.sum(W**2)
            grad = (probs - Y_onehot).T.dot(X) / n_samples + W / (self.C * n_samples)
        elif self.penalty == 'l1':
            loss += 1.0 / (self.C * n_samples) * np.sum(np.abs(W))
            grad = (probs - Y_onehot).T.dot(X) / n_samples + np.sign(W) / (self.C * n_samples)

        return loss, grad.ravel()

    def fit(self, X, y):
        """
        Обучение модели с выбранным solver'ом.
        L-BFGS использует scipy.optimize.minimize,
        SAGA реализован через градиентный спуск с адаптивным шагом.
        """
        X = np.array(X, dtype=float)
        y = np.array(y)
        self.classes_ = np.unique(y)
        n_samples, n_features = X.shape
        n_classes = len(self.classes_)

        Y_onehot = self._one_hot(y)
        W_init = np.zeros((n_classes, n_features))

        if self.solver == 'lbfgs':
            # Оптимизация с использованием L-BFGS
            res = minimize(
                fun=lambda W: self._loss_and_grad(W, X, Y_onehot)[0],
                x0=W_init.ravel(),
                jac=lambda W: self._loss_and_grad(W, X, Y_onehot)[1],
                method='L-BFGS-B',
                options={'maxiter': self.max_iter, 'disp': False}
            )
            self.weights = res.x.reshape(n_classes, n_features)

        elif self.solver == 'saga':
            # Стохастический градиентный спуск с адаптивным шагом
            W = W_init.copy()
            lr = 1.0 / (np.max(np.sum(X ** 2, axis=1)) + 1.0)
            for _ in range(self.max_iter):
                _, grad = self._loss_and_grad(W, X, Y_onehot)
                W -= lr * grad.reshape(n_classes, n_features)
                if np.linalg.norm(lr * grad) < self.tol:
                    break
            self.weights = W

        else:
            raise ValueError("solver must be 'lbfgs' or 'saga'")

    def predict_proba(self, X):
        # Вычисление вероятностей классов через softmax
        X = np.array(X, dtype=float)
        scores = X.dot(self.weights.T)
        return softmax(scores)

    def predict(self, X):
        # Предсказание классов как argmax вероятностей
        probs = self.predict_proba(X)
        return self.classes_[np.argmax(probs, axis=1)]


Обучим на данных бейзлайна

In [81]:
model = MyLogisticRegression()

model.fit(X_c_base_train, y_c_base_train)

y_pred = model.predict(X_c_base_test)

acc = accuracy_score(y_c_base_test, y_pred)

print("Accuracy:", acc)

/tmp/ipykernel_343466/768891952.py:61: DeprecationWarning: scipy.optimize: The `disp` and `iprint` options of the L-BFGS-B solver are deprecated and will be removed in SciPy 1.18.0.
  res = minimize(


Accuracy: 0.3032


Обучим на данных улучшенной модели. Сразу будем использовать параметры, полученные при подборе гиперпараметров

In [82]:
preprocess = ColumnTransformer(
    transformers=[
        ('float_scaling', StandardScaler(), float_features),
    ],
    remainder='passthrough'
)

pipeline = Pipeline([
    ('preprocess', preprocess),
    ('logreg', MyLogisticRegression(penalty='l1', C=0.1, solver='saga'))
])

pipeline.fit(X_c_train, y_c_train)
preds = pipeline.predict(X_c_test)
acc = accuracy_score(y_c_test, preds)
print("Accuracy:", acc)

print(classification_report(y_c_test, preds))

Accuracy: 0.4026
              precision    recall  f1-score   support

           0       0.65      0.02      0.05      1000
           1       0.45      0.55      0.49      1000
           2       0.44      0.20      0.28      1000
           3       0.35      0.95      0.51      1000
           4       0.63      0.13      0.21      1000
           5       0.52      0.34      0.41      1000
           6       0.36      0.69      0.47      1000
           7       0.40      0.20      0.27      1000
           8       0.37      0.29      0.32      1000
           9       0.43      0.66      0.52      1000

    accuracy                           0.40     10000
   macro avg       0.46      0.40      0.35     10000
weighted avg       0.46      0.40      0.35     10000



Получилось хуже, чем у sklearn. Многое можно улучшить

#####  Имплементация линейной регрессии

In [ ]:
class MyLinearRegression(BaseEstimator, RegressorMixin):

    def __init__(self, fit_intercept: bool = True):
        self.fit_intercept = fit_intercept

    def fit(self, X, y):
        """
        Обучение модели
        """

        X = np.asarray(X)
        y = np.asarray(y)

        # Добавляем столбец единиц для intercept при необходимости
        if self.fit_intercept:
            X_design = np.c_[np.ones(X.shape[0]), X]
        else:
            X_design = X

        # МНК
        coef, _, _, _ = np.linalg.lstsq(X_design, y, rcond=None)

        # Сохраняем коэффициенты в формате sklearn
        if self.fit_intercept:
            self.intercept_ = coef[0]
            self.coef_ = coef[1:]
        else:
            self.intercept_ = 0.0
            self.coef_ = coef

        return self  

    def predict(self, X):
        """
        Предсказание
        """
        X = np.asarray(X)

        # y = Xw + b
        return X @ self.coef_ + self.intercept_

Обучим на данных бейзлайна

In [62]:
knn = MyLinearRegression()
knn.fit(X_r_base_train, y_r_base_train)
y_pred = knn.predict(X_r_base_test)

mae = mean_absolute_error(y_r_base_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_r_base_test, y_pred))
r2 = r2_score(y_r_base_test, y_pred)

print("MAE:", mae)
print("RMSE:", rmse)
print("R2:", r2)

MAE: 423673.5471084352
RMSE: 520176.7045925049
R2: 0.16364456646517678


Обучим на данных улучшенной модели.

In [66]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_features),
        ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_store),
        ('passth', 'passthrough', other_feats)
    ],
    remainder='drop'
)

pipeline = Pipeline([
    ('pre', preprocessor),
    ('model', MyLinearRegression())
])

param_grid = {
    'model__fit_intercept': [True, False]
}

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

grid.fit(X_r_train, y_r_train)

print("Best model:", grid.best_estimator_.named_steps['model'])
print("Best params:", grid.best_params_)

y_pred = grid.predict(X_r_test)

mae  = mean_absolute_error(y_r_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_r_test, y_pred))
r2   = r2_score(y_r_test, y_pred)

print(f"\nTest MAE:  {mae:.2f}")
print(f"Test RMSE: {rmse:.2f}")
print(f"Test R2:   {r2:.4f}")

Fitting 5 folds for each of 2 candidates, totalling 10 fits


Best model: MyLinearRegression()
Best params: {'model__fit_intercept': True}

Test MAE:  70854.16
Test RMSE: 109924.95
Test R2:   0.9627


Получилось даже неплохо

| Метрика | Бейзлайн линейной регрессии | Бейзлайн имплементации | Улучшенная линейная рергессия | Улучшенная имплементация |
|-|-|-|-|-|
| MAE | 432655 | 423674 | 70758 | 70854 |
| RMSE | 518509 | 520177 | 109901 | 109925 |
| R2 | 0.129 | 0.164 | 0.963 | 0.963 |